In [1]:
import scipy.stats as stats
import numpy as np
import csv

<h3>ONE MEAN TEST EXAMPLES</h3>

In [2]:
def one_mean_hypothesis_test(
        data_file=None,
        data_summary={"vname":None,"avg":None,"std":None,"size":None},
        pop_normal=True,
        pop_std=None,
        test_param={"alpha":0.05,"H0":0.0,"H0_test":"E"},
    ):
    """
    @param data_file: directory of the data file (default=None)
    @param data_summary: vname, avg, std, and size of data. Note that this param is
        ignored if data file is provided. Defaults: name=None,avg=None,std=None,size=None
    @param pop_normal: boolean, true if popolation values are normally distributed
    @param pop_std: std of population values, None if not known (default=None)
    @param test_param: parameters of the hypothesis tests.
        alpha: type I error (default=0.05)
        H0: value to compare to in the null hypothesis (default=0.0)
        H0_test: 'E' (equal), 'GTE' (greater than or equal to), or 'LTE' (less than or equal to)
            (default='E')
    """
    # Reading data if file is given
    data=[]
    if data_file!=None:
        try:
            with open(data_file) as file:
                reader=csv.reader(file,delimiter=',')
                for index,row in enumerate(reader):
                    if index==0:
                        vname=row[0]
                    else:
                        data.append(float(row[0]))
        except:
            print("Couldn't load file {}. Program terminated!".format(data_file))
            return

    # Else, load data summary from param
    else:
        vname=data_summary.get("vname",None)
        data_avg=data_summary.get("avg",None)
        data_std=data_summary.get("std",None)
        data_size=data_summary.get("size",None)
        data_min=None
        data_max=None
        data_min_strf="%15s"
        data_max_strf="%15s"
        
    # Calculate data summary if data_file is provided
    if data_file!=None:
        data_avg=np.mean(data)
        data_std=np.std(data)
        data_size=len(data)
        data_min=np.min(data)
        data_max=np.max(data)
        data_min_strf="%15.4f"
        data_max_strf="%15.4f"
    
    # Load test params
    test_alpha=test_param.get("alpha",0.05)
    test_H0=test_param.get("H0",0.0)
    test_H0test=test_param.get("H0_test","E")    
    
    # Print variable summary statistics
    if vname==None:
        vname="unknown"
    print("Variable: {}".format(vname))
    print("%10s%15s%15s%15s%15s%15s"%("N","Mean","Std Dev","Std Err","Minimum","Maximum"))
    print("%10d%15.4f%15.4f%15.4f{}{}".format(data_min_strf,data_max_strf)%(
        data_size,
        data_avg,
        data_std,
        data_std/np.sqrt(data_size),
        data_min,
        data_max
    ))
    
    # Case 1: Normal population or large sample size with known std
    if (pop_normal or data_size>=30) and pop_std!=None:
        z_obs=(data_avg-test_H0)/(pop_std/np.sqrt(data_size))
        if test_H0test=='E':
            p_val=2*stats.norm.cdf(-np.absolute(z_obs))
        elif test_H0test=='GTE':
            p_val=stats.norm.cdf(z_obs)
        elif test_H0test=="LTE":
            p_val=1-stats.norm.cdf(z_obs)
        if p_val<=test_alpha:
            dec_str="reject"
        else:
            dec_str="fail to reject"
        
        # Print results
        print("\nThe z-test procedure")
        print("%15s%15s%15s%20s"%("z-Value","p-Value","Type I Err","Decision"))
        print("%15.4f%15.4f%15.4f%20s"%(z_obs,p_val,test_alpha,dec_str))
        return
            
    # Case 2: Normal population or large sample size with unknown std
    if (pop_normal or data_size>=30) and pop_std==None:
        t_obs=(data_avg-test_H0)/(data_std/np.sqrt(data_size))
        if test_H0test=='E':
            p_val=2*stats.t.cdf(-np.absolute(t_obs),df=data_size-1)
        elif test_H0test=='GTE':
            p_val=stats.t.cdf(t_obs,df=data_size-1)
        elif test_H0test=="LTE":
            p_val=1-stats.t.cdf(t_obs,df=data_size-1)
        if p_val<=test_alpha:
            dec_str="reject"
        else:
            dec_str="fail to reject"
        
        # Print results
        print("\nThe t-test procedure")
        print("%10s%15s%15s%15s%20s"%("DF","z-Value","p-Value","Type I Err","Decision"))
        print("%10d%15.4f%15.4f%15.4f%20s"%(data_size-1,t_obs,p_val,test_alpha,dec_str))
        return
        
    # Case 3: Non-normal population and small sample size
    print("\nNon-normally-distributed population values and small sample size. No appropriate test!")
    return

In [3]:
one_mean_hypothesis_test(
    data_file=None,
    data_summary={"vname":"chair_height","avg":37.5,"std":5.0,"size":30},
    pop_normal=True,
    pop_std=None,
    test_param={"H0":40,"H0_test":'E'}
)

Variable: chair_height
         N           Mean        Std Dev        Std Err        Minimum        Maximum
        30        37.5000         5.0000         0.9129           None           None

The t-test procedure
        DF        z-Value        p-Value     Type I Err            Decision
        29        -2.7386         0.0104         0.0500              reject


<h3>DIFFERENCE OF TWO MEANS TEST EXAMPLES</h3>

In [4]:
def two_means_diff_hypothesis_test(
        data_file={"directory":None,"class_name":None,"var_name":None},
        data_summary={
            "var_name":None,
            "s1_vname":None,"s1_avg":None,"s1_std":None,"s1_size":None,
            "s2_vname":None,"s2_avg":None,"s2_std":None,"s2_size":None,
        },
        pop_std=[None,None],
        test_param={"alpha":0.05,"H0_test":"E"},
    ):
    """
    @param data_file: directory, class_name, and var_name
        directory: where the file is stored (default=None)
        class_name: to separate 02 levels in the data (default=None)
        var_name: to provide data values
    @param data_summary: vname, avg, std, and size of data two data samples
    @param pop_std: std of population values of the two samples (default=[None,None])
    @param test_param: parameters of the hypothesis tests.
        alpha: type I error (default=0.05)
        H0: value to compare to in the null hypothesis (default=0.0)
        H0_test: 'E' (equal), 'GTE' (greater than or equal to), or 'LTE' (less than or equal to)
            (default='E')
    """
    # Reading data if file given
    data_file_url=data_file.get("directory",None)
    data_file_cname=data_file.get("class_name",None)
    data_file_vname=data_file.get("var_name",None)
    if data_file and (not data_file_cname) or (not data_file_vname):
        print(
            "Class name or variable name not specified for dataset {}. "
            "Program terminated!".format(data_file_url)
        )
        return
    #
    data={}
    if data_file_url!=None:
        try:
            with open(data_file_url) as file:
                reader=csv.reader(file,delimiter=',')
                for index,row in enumerate(reader):
                    if index==0:
                        cnames=row
                        if data_file_cname not in cnames or data_file_vname not in cnames:
                            print(
                                "Invalid class name or variable name for dataset {}. "
                                "Program terminated!".format(data_file_url)
                            )
                            return
                        else:
                            if row[0]==data_file_cname:
                                cname_idx=0
                                vname_idx=1
                            else:
                                cname_idx=1
                                vname_idx=0
                    else:
                        if row[cname_idx] in data:
                            data[row[cname_idx]].append(float(row[vname_idx]))
                        else:
                            data[row[cname_idx]]=[float(row[vname_idx])]
        except:
            print("Couldn't load file {}. Program terminated!".format(data_file_url))
            return
        if len(data.keys())!=2:
            print("Class name doesn't have exactly 2 levels. Program terminated!")
            return
    
    # Else, load data summary from param
    else:
        data_file_vname=data_summary.get("var_name",None)
        s1_vname=data_summary.get("s1_vname",None)
        s1_avg=data_summary.get("s1_avg",None)
        s1_std=data_summary.get("s1_std",None)
        s1_size=data_summary.get("s1_size",None)
        s2_vname=data_summary.get("s2_vname",None)
        s2_avg=data_summary.get("s2_avg",None)
        s2_std=data_summary.get("s2_std",None)
        s2_size=data_summary.get("s2_size",None)
        data_min_strf="%15s"
        data_max_strf="%15s"
        s1_min=None
        s1_max=None
        s2_min=None
        s2_max=None
    
    # Calculate data summary if data_file is provided
    if data_file_url!=None:
        aux=[key for key in data.keys()]
        if np.mean(data[aux[0]])>np.mean(data[aux[1]]):
            idx1=0
            idx2=1
        else:
            idx1=1
            idx2=0
        s1_vname=aux[idx1]+" "+data_file_cname
        s1_avg=np.mean(data[aux[idx1]])
        s1_std=np.std(data[aux[idx1]])
        s1_size=len(data[aux[idx1]])
        s2_vname=aux[idx2]+" "+data_file_cname
        s2_avg=np.mean(data[aux[idx2]])
        s2_std=np.std(data[aux[idx2]])
        s2_size=len(data[aux[idx2]])
        data_min_strf="%15.4f"
        data_max_strf="%15.4f"
        s1_min=np.min(data[aux[idx1]])
        s1_max=np.max(data[aux[idx1]])
        s2_min=np.min(data[aux[idx2]])
        s2_max=np.max(data[aux[idx2]])
        
    # Print summary statistics
    print("Variable: {}".format(data_file_vname))
    print("%20s%10s%15s%15s%15s%15s%15s"%("","N","Mean","Std Dev","Std Err","Minimum","Maximum"))
    print("%20s%10d%15.4f%15.4f%15.4f{}{}".format(data_min_strf,data_max_strf)%(
        s1_vname,
        s1_size,
        s1_avg,
        s1_std,
        s1_std/np.sqrt(s1_size),
        s1_min,
        s1_max,
    ))
    print("%20s%10d%15.4f%15.4f%15.4f{}{}".format(data_min_strf,data_max_strf)%(
        s2_vname,
        s2_size,
        s2_avg,
        s2_std,
        s2_std/np.sqrt(s1_size),
        s2_min,
        s2_max,
    ))
    
    # Load test params
    test_alpha=test_param.get("alpha",0.05)
    test_H0=test_param.get("H0",0.0)
    test_H0test=test_param.get("H0_test","E")
    
    # Case 1: Known population variances
    if pop_std[0]!=None and pop_std[1]!=None:
        z_obs=(s1_avg-s2_avg-test_H0)/np.sqrt(pop_std[0]**2/s1_size+pop_std[1]**2/s2_size)
        if test_H0test=='E':
            p_val=2*stats.norm.cdf(-np.absolute(z_obs))
        elif test_H0test=='GTE':
            p_val=stats.norm.cdf(z_obs)
        elif test_H0test=="LTE":
            p_val=1-stats.norm.cdf(z_obs)
        if p_val<=test_alpha:
            dec_str="reject"
        else:
            dec_str="fail to reject"
            
        # Print results
        print("\nThe z-test procedure")
        print("%15s%15s%15s%20s"%("z-Value","p-Value","Type I Err","Decision"))
        print("%15.4f%15.4f%15.4f%20s"%(z_obs,p_val,test_alpha,dec_str))
        return
    
    # Case 2: Unknown population variances
    # Assume equal variances
    aux=((s1_size-1)*s1_std**2+(s2_size-1)*s2_std**2)/(s1_size+s2_size-2)
    t_obs1=(s1_avg-s2_avg-test_H0)/np.sqrt(aux/s1_size+aux/s2_size)
    if test_H0test=='E':
            p_val1=2*stats.t.cdf(-np.absolute(t_obs1),df=s1_size+s2_size-2)
    elif test_H0test=='GTE':
        p_val1=stats.t.cdf(t_obs1,df=s1_size+s2_size-2)
    elif test_H0test=="LTE":
        p_val1=1-stats.t.cdf(t_obs1,df=s1_size+s2_size-2)
    if p_val1<=test_alpha:
        dec_str1="reject"
    else:
        dec_str1="fail to reject"
            
    # Not assume equal variances
    t_obs2=(s1_avg-s2_avg-test_H0)/np.sqrt(s1_std**2/s1_size+s2_std**2/s2_size)
    df2=(s1_std**2/s1_size+s2_std**2/s2_size)**2/((s1_std**2/s1_size)**2/(s1_size-1)+
                                                  (s2_std**2/s2_size)**2/(s2_size-1))
    if test_H0test=='E':
        p_val2=2*stats.t.cdf(-np.absolute(t_obs2),df=df2)
    elif test_H0test=='GTE':
        p_val2=stats.t.cdf(t_obs2,df=df2)
    elif test_H0test=="LTE":
        p_val2=1-stats.t.cdf(t_obs2,df=df2)
        
    if p_val2<=test_alpha:
        dec_str2="reject"
    else:
        dec_str2="fail to reject"
        
    # Hypothesis test for the variances
    if s1_std<s2_std:
        F_stats=s1_std**2/s2_std**2
        numDF=s1_size-1
        denDF=s2_size-1
        p_val_f=stats.f.cdf(F_stats,numDF,denDF)
    else:
        F_stats=s2_std**2/s1_std**2
        numDF=s2_size-1
        denDF=s1_size-1
        p_val_f=stats.f.cdf(F_stats,numDF,denDF)
    if p_val_f<=test_alpha:
        dec_str3="reject"
    else:
        dec_str3="fail to reject"
    
    # Print results
    print("\nThe t-test procedure")
    print("%15s%15s%15s%15s%15s%15s%20s"%(
        "Method","Variances","DF","t-Value",
        "p-Value","Type I Err","Decision"
    ))
    print("%15s%15s%15.2f%15.4f%15.4f%15.4f%20s"%(
        "pooled","equal",s1_size+s2_size-2,
        t_obs1,p_val1,test_alpha,dec_str1
    ))
    print("%15s%15s%15.2f%15.4f%15.4f%15.4f%20s"%(
        "satterthwaite","unequal",df2,
        t_obs2,p_val2,test_alpha,dec_str2
    ))
    print("\nEquality of variances")
    print("%15s%15s%15s%15s%15s%15s%20s"%(
        "Method","Num DF","Den DF","f-Value",
        "p-Value","Type I Err","Decision"
    ))
    print("%15s%15.2f%15.2f%15.4f%15.4f%15.4f%20s"%(
        "folded f",numDF,denDF,
        F_stats,p_val_f,test_alpha,dec_str3
    ))
    return

In [5]:
two_means_diff_hypothesis_test(
    data_file={
        "directory":"food_rating.csv",
        "class_name":"hire_decision",
        "var_name":"rating"
    },
    data_summary={
        "var_name":"food rating",
        "s1_vname":"after hire","s1_avg":3.9,"s1_std":0.7,"s1_size":10,
        "s2_vname":"before hire","s2_avg":3.0,"s2_std":1.1,"s2_size":13,
    },
    pop_std=[0.8,None],
    test_param={"alpha":0.05,"H0":1.5,"H0_test":"GTE"},
)

Variable: rating
                             N           Mean        Std Dev        Std Err        Minimum        Maximum
 after hire_decision        13         3.9077         1.1532         0.3198         1.3000         5.0000
before hire_decision        13         3.0538         1.1188         0.3103         1.0000         4.5000

The t-test procedure
         Method      Variances             DF        t-Value        p-Value     Type I Err            Decision
         pooled          equal          24.00        -1.4500         0.0800         0.0500      fail to reject
  satterthwaite        unequal          23.98        -1.4500         0.0800         0.0500      fail to reject

Equality of variances
         Method         Num DF         Den DF        f-Value        p-Value     Type I Err            Decision
       folded f          12.00          12.00         0.9412         0.4591         0.0500      fail to reject
